# Spectrogram

In [ ]:
import matplotlib.pyplot as plt
import subjects
from neuropy import plotting
from neuropy.utils import signal_process
sess = subjects.nsd.ratVday1[0]

In [ ]:
signal = sess.eegfile.get_signal([123])
spect = signal_process.SpectrogramBands(signal, window=10, overlap=5)

In [ ]:
%matplotlib widget
from neuropy.plotting import Fig,plot_epochs

figure = Fig()
fig,gs = figure.draw(grid=(1,1))
ax = plt.subplot(gs[0])
plotting.plot_spectrogram(spect.sxx,spect.time,spect.freq,ax=ax)

# plot_epochs(ax,sess.artifact)

# Theta

## Best theta channels

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from neuropy.utils import signal_process
import subjects

sessions = subjects.sd.ratUday4

In [ ]:
for sub,sess in enumerate(sessions):
    maze = sess.paradigm['maze'].flatten()

    changrp= sess.probegroup.get_connected_channels(groupby='shank')
    bestchans=[]
    for channels in changrp:
        signal = sess.eegfile.get_signal(channel_indx=channels.astype('int'),t_start=maze[1],t_stop=maze[1]+1800)
        auc = signal_process.psd_auc(signal=signal,freq_band=(1,4))
        bestchans.append(channels[np.argmax(auc)])

In [ ]:
%matplotlib widget
from neuropy.plotting import plot_probe

plot_probe(sess.probegroup,annotate_channels=bestchans,channel_id=False)